In [2]:
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd
from IPython.display import JSON
import glob
import os

In [1]:
api_key = ''

In [4]:
channel_ids = ['UCCezIgC97PvUuR4_gbFUs5g',
               # more channels here
               ]

In [5]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

In [6]:
def get_channel_stats(youtube, channel_ids):

    """
    Get channel stats

    Params:
    ------
    youtube: build object of Youtube API
    channel_ids: list of channel IDs

    Returns:
    ------
    dataframe with all channel stats for each channel ID

    """

    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    # loop through items
    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
                }

        all_data.append(data)

    return(pd.DataFrame(all_data))

def get_video_ids(youtube, playlist_id):

    video_ids = []

    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId = playlist_id,
            maxResults = 50,
            pageToken = next_page_token)
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

    return video_ids


def get_video_details(youtube, video_ids):

    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                             }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [7]:
channel_stats = get_channel_stats(youtube, channel_ids)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Corey Schafer,1090000,83902795,231,UUCezIgC97PvUuR4_gbFUs5g


In [8]:
playlist_id = "UUCezIgC97PvUuR4_gbFUs5g"


In [9]:
video_ids = get_video_ids(youtube, playlist_id)


In [10]:
os.chdir('C:/Users/sadik/OneDrive - Pace University/Desktop/Youtube Channel Analysis/Downloads')
video_df = get_video_details(youtube, video_ids)
video_df.to_csv('data1.csv',index=False)


In [16]:

path=os.getcwd()
filenames = glob.glob(path + "/*.csv")
print(filenames)

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame
Download_df = pd.concat(dfs, ignore_index=True)
Download_df.to_csv('Download.csv',index=False)


['C:\\Users\\sadik\\OneDrive - Pace University\\Desktop\\Youtube Channel Analysis\\Downloads\\data1.csv', 'C:\\Users\\sadik\\OneDrive - Pace University\\Desktop\\Youtube Channel Analysis\\Downloads\\data2.csv', 'C:\\Users\\sadik\\OneDrive - Pace University\\Desktop\\Youtube Channel Analysis\\Downloads\\data3.csv', 'C:\\Users\\sadik\\OneDrive - Pace University\\Desktop\\Youtube Channel Analysis\\Downloads\\data4.csv', 'C:\\Users\\sadik\\OneDrive - Pace University\\Desktop\\Youtube Channel Analysis\\Downloads\\data5.csv']
